In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.api import Logit

## Import des donnees:
df = pd.read_csv("billets_complet.csv")        #Dataset d'entrainement
test = pd.read_csv("billets_production.csv")     #Dataset de test             #On verrifira le nom du fichier

test.info()                                          #On verrifira que la colonne id est bien présente

print("\n")

## Entrainement du modele:
# y (authenticite): 
y_billet = df.loc[:, df.columns == "is_genuine"]

# On ne garde que les variables significatives a alpha = 5% dans X_Billet:
X_billet = df[["height_right","margin_low", "margin_up", "length"]]
X_billet = sm.add_constant(X_billet)

# On recommence la regression logistique:
reg_log = Logit(endog=y_billet,exog=X_billet)

# Creation du modele:
model_reg_log = reg_log.fit()

print("\n")

## Prediction sur des donnes inconnues:
# Calculs:
X_test = test[["height_right","margin_low","margin_up","length"]]
X_test = sm.add_constant(X_test)
test["Probabilité vrai billet"] = model_reg_log.predict(X_test)
test["Probabilité vrai billet"] = test["Probabilité vrai billet"]*100
test["Probabilité vrai billet"] = round(test["Probabilité vrai billet"],2)
test["Probabilité vrai billet"] = test["Probabilité vrai billet"].astype(str)+'%'
test["Prédiction"] = (model_reg_log.predict(X_test) >= 0.5).astype(int)
print(test[["id","Probabilité vrai billet","Prédiction"]])

# Affichage des resultats:
print("\n\nIndetification des billets:\n")
for i, k in zip(test["Prédiction"],test["id"]):
    if i == 1:
        print("Le billet","{}".format(k),"est vrai")
    else:
        print("Le billet","{}".format(k),"est faux")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   diagonal      5 non-null      float64
 1   height_left   5 non-null      float64
 2   height_right  5 non-null      float64
 3   margin_low    5 non-null      float64
 4   margin_up     5 non-null      float64
 5   length        5 non-null      float64
 6   id            5 non-null      object 
dtypes: float64(6), object(1)
memory usage: 408.0+ bytes


Optimization terminated successfully.
         Current function value: 0.026815
         Iterations 13


    id Probabilité vrai billet  Prédiction
0  A_1                    0.0%           0
1  A_2                    0.0%           0
2  A_3                    0.0%           0
3  A_4                  99.62%           1
4  A_5                  100.0%           1


Indetification des billets:

Le billet A_1 est faux
Le billet A_2 est faux
Le billet A_3 